In [1]:
%%capture
!pip install rioxarray "xarray[complete]" geopandas


In [2]:
!pip install -U netCDF4 

In [3]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import os
import zarr
import s3fs
from dask.diagnostics import ProgressBar
from datetime import datetime

In [4]:
pathS3 = "s3://climate-action-datalake/zone=raw/source=agera5"
localPath = '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/temporal/*.nc'
outPutPath =  '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/silver/solarFlux.zarr'


In [5]:
def convertRadiation(k):
    c = k/1000000
    return c

In [6]:
allFiles = xr.open_mfdataset(localPath
                             ,parallel=True
                             ,chunks={'time': 50}
                            )

In [7]:
allFiles = allFiles.apply(convertRadiation)

In [8]:
allFiles

<xarray.Dataset> Size: 2TB
Dimensions:               (lat: 2000, lon: 7200, time: 15884)
Coordinates:
  * lat                   (lat) float64 16kB -49.98 -49.93 ... 49.92 49.97
  * lon                   (lon) float64 58kB -180.0 -179.9 ... 179.9 180.0
  * time                  (time) datetime64[ns] 127kB 1980-01-01 ... 2023-06-27
Data variables:
    Solar-Radiation-Flux  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 2000, 7200), meta=np.ndarray>

In [9]:
compressor = zarr.Blosc(cname='lz4')
encoding = {vname: {'compressor': compressor,'chunks': (1,2000,7200)} for vname in allFiles.data_vars}


In [10]:
s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root="s3://climate-action-datalake/zone=raw/source=agera5/variable=solarRadiationFlux.zarr", s3=s3, check=False)

In [ ]:
with ProgressBar():
    result = allFiles.to_zarr(outPutPath
                              , encoding=encoding
                              , consolidated=True
                              ,compute=False
                             )
    
    result.compute(scheduler='processes')

[########################################] | 100% Completed | 2hr 48m


In [ ]:
print("He finalizado")

He finalizado


In [13]:
allFiles

<xarray.Dataset> Size: 2TB
Dimensions:               (lat: 2000, lon: 7200, time: 15884)
Coordinates:
  * lat                   (lat) float64 16kB -49.98 -49.93 ... 49.92 49.97
  * lon                   (lon) float64 58kB -180.0 -179.9 ... 179.9 180.0
  * time                  (time) datetime64[ns] 127kB 1980-01-01 ... 2023-06-27
Data variables:
    Solar-Radiation-Flux  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 2000, 7200), meta=np.ndarray>